<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-df" data-toc-modified-id="Prepare-df-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare df</a></span></li><li><span><a href="#Build-LM-wiki" data-toc-modified-id="Build-LM-wiki-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Build LM wiki</a></span><ul class="toc-item"><li><span><a href="#LM-learner" data-toc-modified-id="LM-learner-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LM learner</a></span><ul class="toc-item"><li><span><a href="#define-learner" data-toc-modified-id="define-learner-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>define learner</a></span></li><li><span><a href="#Start-training" data-toc-modified-id="Start-training-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Start training</a></span></li><li><span><a href="#Save-model" data-toc-modified-id="Save-model-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Save model</a></span></li></ul></li></ul></li><li><span><a href="#Finetuning-LM-with-sentiment-analysis-data" data-toc-modified-id="Finetuning-LM-with-sentiment-analysis-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Finetuning LM with sentiment analysis data</a></span><ul class="toc-item"><li><span><a href="#prepare-datablock-and-dataloaders" data-toc-modified-id="prepare-datablock-and-dataloaders-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>prepare datablock and dataloaders</a></span></li><li><span><a href="#Load-params-and-vocab" data-toc-modified-id="Load-params-and-vocab-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Load params and vocab</a></span></li><li><span><a href="#training" data-toc-modified-id="training-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>training</a></span></li></ul></li><li><span><a href="#Sentiment-classification" data-toc-modified-id="Sentiment-classification-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sentiment classification</a></span><ul class="toc-item"><li><span><a href="#prepare-text-clas-datablock-and-dataloaders" data-toc-modified-id="prepare-text-clas-datablock-and-dataloaders-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>prepare text clas datablock and dataloaders</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="#define-sentiment-clas-learner" data-toc-modified-id="define-sentiment-clas-learner-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>define sentiment clas learner</a></span></li><li><span><a href="#training" data-toc-modified-id="training-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>training</a></span></li></ul></li></ul></li></ul></div>

In [1]:
from fastai.text.all import *

In [2]:
path = Path('/home/quan/.fastai/data/viwiki')

In [3]:
path

Path('/home/quan/.fastai/data/viwiki')

In [4]:
path/'docs'

Path('/home/quan/.fastai/data/viwiki/docs')

In [5]:
# mdl_path = path/'models'
# print(mdl_path)
# mdl_path.mkdir(exist_ok=True)
# lm_fns = [f'vi_wt', f'vi_wt_vocab']

/home/quan/.fastai/data/viwiki/models


In [6]:
(path/'docs').ls()[:5]

(#5) [Path('/home/quan/.fastai/data/viwiki/docs/Doug Emhoff.txt'),Path('/home/quan/.fastai/data/viwiki/docs/HMS Myngs (R06).txt'),Path('/home/quan/.fastai/data/viwiki/docs/Xuân La (phường).txt'),Path('/home/quan/.fastai/data/viwiki/docs/Cung điện Goldstein.txt'),Path('/home/quan/.fastai/data/viwiki/docs/Tứ khố toàn thư.txt')]

Note! Dataset: we have a list of txt files

# Prepare df

In [8]:
def make_lol(path): # make list of list containing file name and file content
    name = path.stem
    with open(path, 'r',encoding="utf-8") as temp:
        data = temp.read()
    return [name,data]

In [9]:
temp = make_lol((path/'docs').ls()[0])
print(temp[1])

Doug Emhoff

Douglas Craig Emhoff (sinh ngày 13 tháng 10 năm 1964) là một luật sư người Mỹ và là chồng của Phó Tổng thống đắc cử Kamala Harris. Ông sẽ trở thành "Đệ nhị Phu quân" đầu tiên trong lịch sử Mỹ và là người phối ngẫu gốc Do Thái đầu tiên của một phó tổng thống.

Emhoff sinh ra tại quận Brooklyn của Thành phố New York, là con trai của bố mẹ gốc Do Thái là bà Barbara (nhũ danh Kanzer) và ông Michael Emhoff. Ông có một anh trai là Andy Emhoff và một em gái là Jamie Emhoff. Ông sống ở New Jersey từ năm 1969 đến năm 1981 trước khi cùng gia đình chuyển đến California năm 17 tuổi. Emhoff lấy bằng Cử nhân Văn học tại Đại học Bang California tại Northridge và bằng Tiến sĩ Luật từ Trường Luật Gould thuộc Viện Đại học Nam California vào năm 1990.

Emhoff đã kết hôn 16 năm với người vợ đầu là bà Kerstin Emhoff (nhũ danh Mackin). Họ có hai con, Cole và Ella. Sau khi cuộc hôn nhân thứ nhất kết thúc, ông kết hôn với bà Kamala Harris vào ngày 22 tháng 8 năm 2014, tại Santa Barbara, Californi

In [10]:
len((path/'docs').ls()) # number of files

82346

In [12]:
# Note: doing things in parallel
??parallel

In [23]:
# results = parallel(make_lol,(path/'docs').ls())

# result_df = pd.DataFrame(results,columns=['fname','text'])

In [25]:
# result_df.tail()

,fname,text
82341,Vấn đề môi trường ở Indonesia,"Vấn đề môi trường ở Indonesia\n\nthumb|360px|Các nhân viên cứu hỏa Indonesia đang cố gắng để ngăn chặn lửa rừng ở Nam Kalimantan (2015).\n\nCác vấn đề môi trường ở Indonesia liên quan đến mật độ dân số cao và công nghiệp hóa nhanh, và chúng thường được ưu tiên thấp hơn do mức nghèo đói cao và quản lý nguồn lực có hạn chế.\n\nCác vấn đề bao gồm nạn phá rừng quy mô lớn (phần lớn là bất hợp pháp) và các vụ cháy rừng liên quan gây ra sương mù dầy đặc ở các khu vực phía Tây Indonesia, Malaysia và Singapore; khai thác quá mức các nguồn tài nguyên biển; và các vấn đề về môi trường liên quan đến q..."
82342,Cát Vinh,"Cát Vinh\n\nCát Vinh (, ? – 528) thủ lĩnh khởi nghĩa nông dân Hà Bắc, là lực lượng lớn mạnh nhất trong phong trào Lục Trấn khởi nghĩa phản kháng nhà Bắc Ngụy.\n\nBan đầu Cát Vinh là Trấn tướng của trấn Hoài Sóc. Theo phép của nhà Bắc Ngụy, Trấn tướng đều lấy quý tộc Tiên Ti đảm nhiệm, từ đó suy đoán ông cũng là người Tiên Ti.\n\nTháng giêng năm Hiếu Xương thứ 2 (526), nguyên Hoài Sóc trấn binh Tiên Vu Tu Lễ tại thành Tả Nhân, Định Châu lãnh đạo khởi nghĩa. Hoài Sóc trấn tướng Cát Vinh tham gia khởi nghĩa.\n\nTháng 8, Nguyên Hồng Nghiệp làm phản, giết Tiên Vu Tu Lễ, xin hàng triều đình. Cát..."
82343,Pye Wacket,"Pye Wacket\n\nPye Wacket là mật danh của một tên lửa không đối không dạng thấu kính thử nghiệm được chi nhánh Convair của tập đoàn General Dynamics phát triển vào năm 1957. Dự định là một tên lửa phòng thủ cho máy bay ném bom B-70 Valkyrie Mach 3, chương trình đã cho thấy thử nghiệm đường ống gió khí động học rộng rãi và có vẻ đầy hứa hẹn; tuy nhiên việc hủy bỏ B-70 đã loại bỏ yêu cầu đối với tên lửa, và dự án này cũng bị hủy bỏ luôn.\n\nDự án ""Pye Wacket"", được chính thức gọi là Chương trình Lenticular Defense Missile (LDM) và theo số dự án WS-740A, được thành lập vào năm 1958 để đáp ứng ..."
82344,Giải cờ vua M-Tel,"Giải cờ vua M-Tel\n\nGiải cờ vua M-Tel, tên chính thức tiếng Bulgaria: М-Тел Мастърс, tiếng Anh M-Tel Masters là một giải cờ vua thường niên được tổ chức từ năm 2005 đến 2009 tại Sofia, thủ đô của Bulgaria. Giải đấu lấy tên theo nhà tài trợ và cũng là nhà tổ chức, công ty mạng di động hàng đầu của Bulgaria M-Tel. Mỗi năm giải mời 6 đại kiện tướng hàng đầu (thường có hệ số điểm Elo trên 2700) tham dự, thi đấu theo thể thức vòng tròn 2 lượt. Địa điểm thi đấu tại khách sạn 5 sao Grand Hotel Sofia. Do những khó khăn về kinh tế, cũng như một số lý do khách quan nên giải đấu không còn được tổ ch..."
82345,Salzburg,"Salzburg\n\nSalzburg ( ; ; nghĩa đen là ""Salt Fortress"" hay ""Pháo đài muối""; tiếng Bayern: ""Soizbuag"") là thủ phủ của tiểu bang cùng tên thuộc Cộng hòa Áo. Với 150.269 dân cư, Salzburg là thành phố lớn thứ tư của Áo sau Viên, Graz và Linz.\n\nThị trấn nằm trên địa điểm của khu định cư La Mã trước đây của ""Iuvavum"". Salzburg được thành lập như một tòa giám mục vào năm 696 và trở thành trụ sở của tổng giám mục vào năm 798. Nguồn thu nhập chính của nó là khai thác và buôn bán muối và đôi khi là khai thác vàng. Pháo đài Hohensalzburg, một trong những pháo đài thời trung cổ lớn nhất ở châu Âu, ..."


In [27]:
# result_df.to_csv(path/'alltext.csv',index=False)

# Build LM wiki

In [11]:
# result_df = pd.read_csv(path/'alltext.csv',encoding='utf-8')
result_df = pd.read_csv(path/'alltext.csv',encoding='utf-8')

In [12]:
result_df.shape

(82346, 2)

In [13]:
vnmese_lm = DataBlock(blocks=TextBlock.from_df('text', is_lm=True),
                    get_x=ColReader('text'),
                    splitter=RandomSplitter())

In [14]:
# bs=64
# bs=24
# bs=128
bs=256

In [15]:
dls = vnmese_lm.dataloaders(result_df, bs=bs, seq_len=72)


/home/quan/anaconda3/envs/fastai_v2/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## LM learner

### define learner

In [18]:
learn = language_model_learner(dls, 
                               AWD_LSTM, 
                               metrics=[accuracy, Perplexity()],
                               drop_mult=1.0,
                               pretrained=False,
                               path=path).to_fp16()

In [19]:
lr = 1e-2
lr *= bs/48
lr

0.05333333333333333

In [20]:
type(learn),type(learn.model)

(fastai.text.learner.LMLearner, fastai.text.models.core.SequentialRNN)

In [21]:
len(learn.model)

2

In [22]:
# learn.model[0],learn.model[1]

In [23]:
learn.unfreeze() # == learn.opt.freeze_to(0)

### Start training

In [24]:

learn.fit_one_cycle(11, lr, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.739719,3.662289,0.338946,38.950413,56:04
1,3.800560,3.711613,0.335228,40.919773,56:04
2,3.706933,3.631249,0.342857,37.759949,55:37
3,3.613249,3.568112,0.350157,35.449596,55:10
4,3.548833,3.499429,0.356900,33.096539,55:14
5,3.498598,3.421560,0.364736,30.617142,55:24
6,3.448882,3.324600,0.375716,27.787888,55:16
7,3.312769,3.221702,0.386235,25.070751,55:11
8,3.238319,3.113604,0.398766,22.501991,55:03
9,3.178566,3.021445,0.409993,20.520914,55:10


In [96]:

# learn.fit_one_cycle(1, lr, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.169965,6.045080,0.118188,422.031403,02:56


In [31]:

# learn.fit_one_cycle(1, lr, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.582673,3.351336,0.375087,28.540852,53:39


In [89]:

# learn.fit_one_cycle(3, lr, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.727336,6.712237,0.097212,822.408630,01:02
1,6.591891,6.539766,0.097202,692.124817,01:01
2,6.492364,6.423313,0.097212,616.040466,01:02


In [11]:

# learn.fit_one_cycle(3, lr, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.153887,5.969858,0.119671,391.450165,02:54
1,4.682713,4.455638,0.261569,86.111069,02:50
2,4.234314,4.185904,0.289397,65.752914,02:49


### Save model

In [33]:
mdl_path = path/'models_2'
print(mdl_path)
mdl_path.mkdir(exist_ok=True)

/home/quan/.fastai/data/viwiki/models_2


In [34]:
mdl_path

Path('/home/quan/.fastai/data/viwiki/models_2')

In [32]:
lm_fns

['vi_wt', 'vi_wt_vocab']

In [28]:
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False) 
# save the state_dict (all possible learnable parameters)

Path('/home/quan/.fastai/data/viwiki/models_2/vi_wt.pth')

In [29]:
if os.path.isfile(mdl_path/(lm_fns[1] + '.pkl')):
    os.remove(mdl_path/(lm_fns[1] + '.pkl'))
    print(True)

In [35]:
len(learn.dls.vocab)

60008

In [31]:
with open(mdl_path/(lm_fns[1] + '.pkl'), 'wb') as f:
    pickle.dump(learn.dls.vocab, f) # save list of strings, which are vocabs

# Finetuning LM with sentiment analysis data

## prepare datablock and dataloaders

In [38]:
train_df = pd.read_csv('train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'

In [39]:
test_df = pd.read_csv('test.csv')
test_df.loc[pd.isna(test_df.comment),'comment']='NA'

In [40]:
df = pd.concat([train_df,test_df], sort=False)
df.head()

,id,comment,label
0,train_000000,Dung dc sp tot cam on \nshop Đóng gói sản phẩm rất đẹp và chắc chắn Chất lượng sản phẩm tuyệt vời,0.0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng khi đánh lên không như màu trên ảnh,0.0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp k có dây giày đen k có tất,0.0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọng cuốn sách khá nhiều hi vọng nó sẽ nói về việc học tập của cách sinh viên trường Harvard ra sao những nỗ lực của họ như thế nào 4h sáng? tại sao họ lại phải thức dậy vào thời khắc đấy? sau đó là cả một câu chuyện ra sao. Cái mình thực sự cần ở đây là câu chuyện ẩn dấu trong đó để tự bản thân mỗi người cảm nhận và đi sâu vào lòng người hơn. Còn cuốn sách này chỉ đơn thuần là cuốn sách dạy kĩ năng mà hầu như sách nào cũng đã có. BUồn...,1.0
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đợt này lại giao 2 cái áo gió chất khác như vải mưa ý :((,1.0


In [41]:
dls_sen = TextDataLoaders.from_df(df, path=path, text_col='comment', 
                                  is_lm=True, 
                                  valid_pct=0.1,
                                  seed=42)

/home/quan/anaconda3/envs/fastai_v2/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## Load params and vocab

In [42]:
lm_fns

['vi_wt', 'vi_wt_vocab']

In [43]:
[mdl_path/(lm_fns[0]+'.pth'),mdl_path/(lm_fns[1] + '.pkl')]


[Path('/home/quan/.fastai/data/viwiki/models/vi_wt.pth'),
 Path('/home/quan/.fastai/data/viwiki/models/vi_wt_vocab.pkl')]

Recreating def load_pretrained

In [44]:
wgts = torch.load(mdl_path/(lm_fns[0]+'.pth'), map_location = lambda storage,loc: storage)

In [45]:
wgts['0.encoder.weight'].shape,wgts['0.rnns.0.weight_hh_l0_raw'].shape

(torch.Size([60008, 400]), torch.Size([4608, 1152]))

In [46]:
learn_sen = language_model_learner(dls_sen, 
                               AWD_LSTM, 
                               metrics=[accuracy, Perplexity()],
                               drop_mult=1.0,
                               pretrained_fnames=lm_fns,
                               path=path).to_fp16()

In [47]:
# bs=64
# bs=24
bs=256

In [48]:
lr = 1e-3
lr *= bs/48.

In [49]:
len(dls_sen.vocab)

5128

In [50]:
learn_sen.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(5128, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(5128, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=5128, bias=True)
    (output_dp): RNNDropout()
  )
)

In [51]:
[f"{getattr(i,'shape')}: {i.requires_grad}" for l in learn_sen.opt.param_lists for i in l]
# only the head (last linear layer) is True, since we only finetune the head

['torch.Size([4608, 1152]): False',
 'torch.Size([4608, 400]): False',
 'torch.Size([4608]): False',
 'torch.Size([4608]): False',
 'torch.Size([4608, 1152]): False',
 'torch.Size([4608, 1152]): False',
 'torch.Size([4608]): False',
 'torch.Size([4608]): False',
 'torch.Size([1600, 400]): False',
 'torch.Size([1600, 1152]): False',
 'torch.Size([1600]): False',
 'torch.Size([1600]): False',
 'torch.Size([5128, 400]): True',
 'torch.Size([5128]): True']

## training

In [52]:
learn_sen.fit_one_cycle(3, lr*10, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.561561,4.085917,0.320092,59.496471,00:15
1,4.162527,3.937553,0.332838,51.292919,00:16
2,4.023841,3.903518,0.335077,49.576576,00:15


In [53]:
learn_sen.unfreeze()
learn_sen.fit_one_cycle(8, lr)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.917189,3.817052,0.343262,45.469982,00:16
1,3.801542,3.710258,0.354930,40.864330,00:16
2,3.702930,3.646227,0.361281,38.329762,00:16
3,3.595436,3.604486,0.366920,36.762775,00:16
4,3.552714,3.580808,0.370587,35.902554,00:16
5,3.488484,3.566941,0.371994,35.408108,00:16
6,3.455432,3.563009,0.373032,35.269146,00:16
7,3.429385,3.562340,0.373032,35.245586,00:16


In [22]:
# learn_sen.fit_one_cycle(3, lr*10, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.629779,4.114153,0.319868,61.200378,00:33
1,4.226692,3.956623,0.331434,52.280468,00:32
2,4.043056,3.915927,0.335153,50.195576,00:32


In [23]:
# learn_sen.unfreeze()
# learn_sen.fit_one_cycle(8, lr)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.970504,3.848961,0.341873,46.944244,00:36
1,3.876658,3.764037,0.350300,43.122166,00:36
2,3.775550,3.710275,0.355639,40.865040,00:37
3,3.702173,3.653225,0.362710,38.598965,00:36
4,3.634814,3.627688,0.365209,37.625713,00:36
5,3.563126,3.611554,0.367025,37.023540,00:36
6,3.509881,3.606653,0.367896,36.842525,00:36
7,3.499975,3.606580,0.367998,36.839863,00:36


In [54]:
learn_sen.save_encoder('finetuned_sen')

# Sentiment classification

TODO: Check how word embedding and hidden state are loaded to sentiment model

In [ ]:
# or this https://medium.com/@hiromi_suenaga/deep-learning-2-part-2-lesson-10-422d87c3340c
# https://nlp.fast.ai/
# watch https://www.youtube.com/watch?v=vnOpEwmtFJ8

## prepare text clas datablock and dataloaders

In [55]:
# bs=64
# bs=24
bs=256

In [56]:
train_df = pd.read_csv('train.csv')

In [57]:
train_df.loc[pd.isna(train_df.comment),'comment']='NA'

In [58]:
train_df.head()

,id,comment,label
0,train_000000,Dung dc sp tot cam on \nshop Đóng gói sản phẩm rất đẹp và chắc chắn Chất lượng sản phẩm tuyệt vời,0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng khi đánh lên không như màu trên ảnh,0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp k có dây giày đen k có tất,0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọng cuốn sách khá nhiều hi vọng nó sẽ nói về việc học tập của cách sinh viên trường Harvard ra sao những nỗ lực của họ như thế nào 4h sáng? tại sao họ lại phải thức dậy vào thời khắc đấy? sau đó là cả một câu chuyện ra sao. Cái mình thực sự cần ở đây là câu chuyện ẩn dấu trong đó để tự bản thân mỗi người cảm nhận và đi sâu vào lòng người hơn. Còn cuốn sách này chỉ đơn thuần là cuốn sách dạy kĩ năng mà hầu như sách nào cũng đã có. BUồn...,1
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đợt này lại giao 2 cái áo gió chất khác như vải mưa ý :((,1


In [59]:
train_df.label.value_counts(True)

0    0.576863
1    0.423137
Name: label, dtype: float64

In [60]:
sen_db = DataBlock(blocks=(TextBlock.from_df('comment', seq_len=72, vocab=dls_sen.vocab), CategoryBlock),
                      get_x=ColReader('text'), # fixed value
                      get_y=ColReader('label'),
                      splitter=RandomSplitter(valid_pct=0.1))

In [61]:
dls_clas = sen_db.dataloaders(train_df, bs=bs)

/home/quan/anaconda3/envs/fastai_v2/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [62]:
dls_clas.before_batch # maybe in text dataloader definition

Pipeline: Pad_Chunk -- {'pad_idx': 1, 'pad_first': True, 'seq_len': 72}

In [72]:
# dls_clas = TextDataLoaders.from_df(train_df, path=path, text_col='comment', 
#                                    text_vocab = dls_sen.vocab,
#                                    label_col='label',
#                                   valid_pct=0.1,
#                                   seed=42)

In [130]:
# dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj cuốn này mỏng . đọc một buổi sáng là hết . xxmaj thú thật nó làm tôi hơi thất vọng có thể vì tôi đã kì vọng hơi cao vào tác phẩm xxmaj hàn xxmaj quốc đầu tiên trong đời . xxmaj nhưng thôi tôi sẽ cố gắng nhận xét khách quan nhất có thể . \n xxmaj tuy đặt vào tình huống đau buồn - sau sự ra đi của một cậu học sinh mới mười sáu tuổi - nhưng xxmaj tôi đã chết vào một ngày nào đó không quá u ám nặng nề . xxmaj thay vì khai thác tột cùng nỗi đau và mất mát , tác giả cố gắng xxunk dịu nó , hàn gắn nó bằng những kí ức tươi đẹp , những suy tư giản dị , gần gũi và những tình cảm chân thành . xxmaj truyện kể theo lời",1
1,"xxbos xxmaj cuốn sách này của xxup tờ xxmaj pi mình không mua mà đọc được nó khi một người bạn giới thiệu và cho mượn .khi đọc xong cuốn sách này thì mình cảm thấy một điều là chưa đủ . \n xxmaj tác giả chưa đủ trải , câu chuyện chưa đủ ngấm . xxmaj các bạn trẻ khi đọc vào có thể cảm thấy , ừ tác phẩm này rất đời , rất thực ấy chứ . xxmaj nhưng với mình như thế vẫn là chưa đủ cho một tác phẩm mang tính phản ánh xã hội . xxmaj khi mà tác giả mượn những câu chuyện thường ngày , mượn những điều gần gũi , chân thực với mỗi chúng ta để xxunk lộ , cắt lát nhiều khía cạnh cuộc sống thì mình thấy tác giả phải đủ trải hay ít nhất là đủ hiểu",1
2,"xxbos xxmaj mình đặt mua xxmaj tạm biệt em ổn hộ một người bạn . xxup nó siêu thích chị xxup tờ xxmaj pi vì có theo dõi facebook chị ý nó bảo chị ý cá tính status nhiều câu hay và đúng xxunk về , mình có tò mò đọc thử , trước đó mình chưa từng đọc gì liên quan đến chị xxup tờ xxmaj pi cả . \n xxmaj với quan điểm cá nhân của mình , thì mình không ưng cuốn sách ở lối viết . xxup vì mình hoài cổ lắm , mình thích văn học nhẹ nhàng , bay bổng , mà chị xxup tờ xxmaj pi viết thì trần trụi quá , đúng kiểu văn thô mới là văn thật . \n xxmaj trong xxmaj tạm biệt , em ổn là những câu chuyện rất thật , rất đời . xxmaj tuy không",1


## Training

### define sentiment clas learner

In [63]:
learn_clas = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                     metrics=[accuracy,F1Score()],
                                    seq_len=72,
                                    path=path)

In [64]:
learn_clas.path

Path('/home/quan/.fastai/data/viwiki')

In [65]:
learn_clas = learn_clas.load_encoder('finetuned_sen')

In [66]:
lr=2e-2
lr *= bs/48
lr

0.10666666666666666

In [67]:
type(learn_clas),type(learn_clas.model),len(learn_clas.model) # no more LMLearner

(fastai.text.learner.TextLearner, fastai.text.models.core.SequentialRNN, 2)

In [196]:
learn_clas.model[0] # same as LMLearner finetuning

# The model uses a SentenceEncoder, 
# which means the texts are passed seq_len tokens at a time, 
# and WILL ONLY COMPUTE THE GRADIENTS on the last max_len steps (default to 1440)

# This will take care of the question: some reviews are long, how to know we have 
# fetched an entire review before gradient can be computed?

SentenceEncoder(
  (module): AWD_LSTM(
    (encoder): Embedding(5128, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(5128, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
)

In [198]:
learn_clas.model[1] # not linear decoder
# 1200 is 400*3, or (the size of hidden state activation of last LSTM) * 3
# which includes: [last_hidden, max_pool, avg_pool]
# learn how that is calculated: https://docs.fast.ai/text.models.core#masked_concat_pool

PoolingLinearClassifier(
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.2, inplace=False)
      (2): Linear(in_features=1200, out_features=50, bias=False)
      (3): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): Linear(in_features=50, out_features=2, bias=False)
    )
  )
)

### training

In [68]:
learn_clas.fit_one_cycle(3, lr)

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.391315,0.309082,0.863806,0.855826,00:05
1,0.394070,0.264454,0.887438,0.870251,00:05
2,0.359406,0.260870,0.889304,0.871014,00:05


In [69]:
learn_clas.save(f'vi_clas')

Path('/home/quan/.fastai/data/viwiki/models/vi_clas.pth')

In [70]:
learn_clas.freeze_to(-2)
learn_clas.fit_one_cycle(2, slice(lr/(2.6**4),lr))

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.333976,0.313808,0.881841,0.872140,00:05
1,0.276791,0.238934,0.902363,0.884813,00:05


In [71]:
learn_clas.save(f'vi_clas')

Path('/home/quan/.fastai/data/viwiki/models/vi_clas.pth')

In [72]:
learn_clas.freeze_to(-3)
learn_clas.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2))

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.237310,0.287528,0.900497,0.888112,00:06
1,0.198763,0.275911,0.904851,0.887748,00:06


In [88]:
learn_clas.save(f'vi_clas')

In [82]:
# learn_clas.fit_one_cycle(3, lr)

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.389166,0.272311,0.876244,0.867067,00:07
1,0.361879,0.259818,0.891169,0.881195,00:07
2,0.373139,0.248702,0.896144,0.883947,00:07


In [84]:
# learn_clas.save(f'vi_clas')

In [85]:
# learn_clas.freeze_to(-2)
# learn_clas.fit_one_cycle(2, slice(lr/(2.6**4),lr))

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.332855,0.263983,0.878731,0.852608,00:08
1,0.236296,0.214972,0.909826,0.896650,00:07


In [86]:
learn_clas.freeze_to(-3)
learn_clas.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2))

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.259469,0.235889,0.910448,0.900690,00:11
1,0.201866,0.244321,0.913557,0.903001,00:11


In [88]:
# learn_clas.save(f'vi_clas')

In [89]:
# learn_clas.unfreeze()
# learn_clas.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10))

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.161196,0.448453,0.907960,0.896648,00:16
